In [1]:
# Imports
import os
import tweepy as tw
from tweepy import TweepError
import pandas as pd
import numpy as np
import webbrowser
import datetime
import schedule
import time
import glob
from IPython.display import clear_output
from time import sleep
import urllib3.contrib.pyopenssl
urllib3.contrib.pyopenssl.inject_into_urllib3()

In [2]:
# Für Twitter API --> Developer Account
consumer_key= ''
consumer_secret= ''
access_token= ''
access_token_secret= ''

In [3]:
# Eigene Methoden für das Erstellen von Dateien --> Methoden sagen eigentlich schon aus, was gemacht wird
# Ratenlimitierung der Twitter API wird im Auge gehalten
def test_rate_limit(api, wait=True, buffer=.1):
    try:
        remaining = int(api.last_response.headers['x-rate-limit-remaining'])
        print("Remaining:", remaining)
        limit = int(api.last_response.headers['x-rate-limit-limit'])
        print("Limit: ", limit)
        reset = int(api.last_response.headers['x-rate-limit-reset'])
        resetdate = datetime.datetime.fromtimestamp(reset)
        print("Reset: ", resetdate)
        return True
    except:
        print()
        
# Sucht Tweet
def searchTwitterAPI(keyword, since, until, quantity = None, language="de"):
    if quantity == None:
        tweets = tw.Cursor(api.search, q="{} since:{}, until:{}".format(keyword, since, until), count=100, result_type="recent", lang=language, tweet_mode="extended").items()
        #tweets = tw.Cursor(api.search, q="{} since:{}, until:{}, exclude:retweets".format(keyword, since, until), count=100, result_type="recent", lang=language, tweet_mode="extended").items()
    else:
        tweets = tw.Cursor(api.search, q="{} since:{}, until:{}".format(keyword, since, until), count=100, result_type="recent", lang=language, tweet_mode="extended").items(quantity)
        #tweets = tw.Cursor(api.search, q="{} since:{}, until:{}, exclude:retweets".format(keyword, since, until), count=100, result_type="recent", lang=language, tweet_mode="extended").items(quantity)
    tweetlist = []
    for i, tweet in enumerate(tweets):
        print("Aktuelles Keyword: ", keyword)
        print("Anzahl: ", i)
        test_rate_limit(api)
        clear_output(wait=True)
        tweetlist.append(tweet)
    print("Transforming to json")
    json_data = [r._json for r in tweetlist]
    return pd.json_normalize(json_data)

# Liest die Keywordliste
def readKeywordFile(file):
    keywords = []
    with open(file, "r") as filestream:
        for i, line in enumerate(filestream):
            currentline = line.split(",")
            for eintrag in currentline:
                keywords.append(eintrag.replace(" ", "").replace("\n", "").replace("ae", "ä").replace("oe", "ö").replace("ue", "ü"))
    return list(filter(None, keywords))

# Holt sich alle Tweets für einen Tag mit der Keywordliste
def job_EveryDay(keywords):
    schedule.clear()
    clear_output(wait=True)
    today = datetime.datetime.now().strftime("%Y-%m-%d")
    yesterday = (datetime.datetime.now() - pd.to_timedelta(1,'d')).strftime("%Y-%m-%d")
    if not os.path.exists("../Daten/TwitterApi/{}".format(yesterday)):
        os.mkdir("../Daten/TwitterApi/{}".format(yesterday))
    for i, keyword in enumerate(keywords):
        #filename = "{}_{}".format(yesterday, i) # Nummer
        filename = "{}_{}".format(yesterday, keyword)  # Keyword
        searchTwitterAPI(keyword=keyword, since=yesterday, until=today).to_csv("../Daten/TwitterApi/{}/{}.csv".format(yesterday, filename), sep = ';', line_terminator = '', encoding = 'utf-8')
    print("Ende")

# Holt sich von einen bestimmten Tag mit einer bestimmten Keywordliste Tweets    
def getTwitterFromDay(date, keywords):
    clear_output(wait=True)
    yesterday = datetime.datetime.strptime(date, "%Y-%m-%d").strftime("%Y-%m-%d")
    till = (datetime.datetime.strptime(date, "%Y-%m-%d") + pd.to_timedelta(1,'d')).strftime("%Y-%m-%d")
    #print("Daten zwischen: ", yesterday, " : ", till)
    if not os.path.exists("../Daten/TwitterApi/{}".format(yesterday)):
        os.mkdir("../Daten/TwitterApi/{}".format(yesterday))
    for i, keyword in enumerate(keywords):
        clear_output(wait=True)
        print("Daten zwischen: ", yesterday, " : ", till)
        print("Akutelles Keyword: ", keyword)
        #filename = "{}_{}".format(yesterday, i) # Nummer
        filename = "{}_{}".format(yesterday, keyword)  # Keyword
        searchTwitterAPI(keyword=keyword, since=yesterday, until=till).to_csv("../Daten/TwitterApi/{}/{}.csv".format(yesterday, filename), sep = ';', line_terminator = '', encoding = 'utf-8')

# Holt sich in einem Zeitraum Tweets mit einer Keywordliste
def getTwitterBetweenDates(fromDate, tillDate, keywords):
    fromThisDate = datetime.datetime.strptime(fromDate, "%Y-%m-%d")
    toThisDate = datetime.datetime.strptime(tillDate, "%Y-%m-%d")
    listWithDates = pd.date_range(fromThisDate,(toThisDate-pd.to_timedelta(1,'d')),freq='d')
    #print(listWithDates[-1:].strftime("%Y-%m-%d"))
    for date in listWithDates:
        getTwitterFromDay(date.strftime("%Y-%m-%d"), keywords)
    print("Ende")

In [5]:
keywords = readKeywordFile("./KeywordsGER.txt")

OAuth 2 Authentication (Limit: 450 x 100 pro 15min).


Nur die letzten 7 Tage sind hiermit möglich!

In [6]:
auth = tw.AppAuthHandler(consumer_key, consumer_secret)
api = tw.API(auth, wait_on_rate_limit=True, retry_count=5,retry_errors=set([401, 404, 500, 503]))

In [7]:
job_EveryDay(keywords)

Transforming to json
Ende


In [ ]:
# Jeden Tag selbstständig Daten holen
schedule.every().day.at("09:00").do(job_EveryDay, keywords)
while True:
    schedule.run_pending()
    time.sleep(60) # Check every minute

In [2]:
#getTwitterFromDay("2021-03-23", keywords[60:])

In [3]:
#getTwitterBetweenDates("2021-03-15", "2021-03-18", keywords)